In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib      # pip install matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive
from IPython.display import display
import folium

In [ ]:
def table_clean():
    df_hosp_beds = pd.read_csv('https://raw.githubusercontent.com/healthbiodatascientist/Health-Dash/refs/heads/main/beds_by_nhs_board-of-treatment_specialty.csv') # Open Public Health Scotland Hospital Beds Data
    df_map = gpd.read_file('https://github.com/healthbiodatascientist/Health-Dash/raw/refs/heads/main/SG_NHS_HealthBoards_2019.shp') # Scottish Health Board Region Names and Geometries
    df_map = df_map.filter(items=['HBCode', 'HBName', 'geometry']) # filter for the columns we need
    df_map = df_map.set_index('HBCode') # set the index to the Health Board Code
    df_hb_beds = df_map.join(df_hosp_beds.set_index('HB'), on='HBCode') # Join the tables
    df_hb_beds = df_hb_beds.filter(items=['FinancialYear', 'SpecialtyName', 'HBName', 'PercentageOccupancy', 'AverageAvailableStaffedBeds', 'AllStaffedBeds', 'geometry']) # filter
    return df_hb_beds
df_hb_beds = table_clean()

In [ ]:
years = np.unique(df_hb_beds['FinancialYear'].values) # setup the values for the filters
specialties = np.unique(df_hb_beds['SpecialtyName'].values)
data_categories = df_hb_beds.columns[3:6]

In [ ]:
def filter_table(year, specialty, data_category):
    # filters the table based on the dropdowns and produces the map
    df_hb_beds = table_clean()
    df_hb_beds = df_hb_beds.loc[df_hb_beds['FinancialYear'].str.startswith(year, na=False)] # filter for year
    df_hb_beds = df_hb_beds.loc[df_hb_beds['SpecialtyName'] == specialty] # filter for specialism
    df_hb_beds = df_hb_beds.filter(items= ['HBName', data_category, 'geometry'])
    foliummap = folium.Map(location=[55.941457, -3.205744], zoom_start=6)
    mapped = df_hb_beds.explore(m=foliummap, column='HBName')
    mapped.save('foliummap.html')
    return mapped

In [ ]:
dropdowns = interactive(filter_table, year=years, specialty=specialties, data_category=data_categories) # run the function above
display(dropdowns)

In [ ]:
dropdowns.result